# CH8 Implémentation des graphes. 

On peut implémenter les graphes.
- par liste d'adjacence (on utilise alors un dictionnaire Python)
- par matrice d'adjacence (on adjoint à une liste de sommets un tableau à deux dimensions )


## I) préambule : affichage des graphes 

Les deux fonctions `affiche_mat` et `affiche_adj` permettent d'afficher dans un fichier `.png` les graphes créés respectivement par matrice d'adjacence et par liste d'adjacence. 
Pour visualiser les fichiers ainsi créés, il faut ensuite ajouter une cellule de type Markdown et intégrer une balise html d'affichage d'image ex :  `<img src="image.png">`
<br>On n'est pas obligé d'étudier dans le détail ces fonctions qui supposent la connaissance du module graphviz, mais on prêtera attention aux docstrings
<br>

Installation de graphviz (à ne faire qu'une fois dans un prompt Anaconda ):<br>
`set https_proxy=https://10.0.0.1:3128  # cas où il y a un proxy
 pip install graphviz`

Attention, le module graphviz nécessite l'installation du logiciel graphviz :
- sur windows : ` https://www2.graphviz.org/Packages/stable/windows/10/msbuild/Release/Win32/`
- sur linux : `sudo apt-get install graphviz`


In [4]:
# Chargement du module graphique
from graphviz import Graph, Digraph

In [14]:
def affiche_mat(gr,nom,orient):
    """
    entree : gr (dict) un graphe défini par matrice
                 ce graphe est un dictionnaire contenant 2 clés
                 "sommets" a pour valeur la liste des sommets
                 "adjacence" est la matrice d'adjacence
             nom (str) le noim du fichier png créé
             orient (bool) le graphe est-il orienté ?
    sortie : aucune, mais création d'un fichier nom.png
    """
    if orient :
        g = Digraph(format='png')
    else:
        g = Graph(format='png')
    for s in gr["sommets"]: # sommets
        g.node(s,s)
    n=len(gr["sommets"])
    for i in range(n) :
        for j in range(n):
            if gr["adjacence"][i][j]==1:
                if orient or i<=j : # astuce pour éviter les doublons dans les arêtes 
                    g.edge(gr["sommets"][i],gr["sommets"][j])
    g.render(nom)

In [1]:
def affiche_adj(gr,nom,orient):
    """
    entree : gr (dict) un graphe défini par liste d'adjacence
             nom (str) le noim du fichier png créé
             orient (bool) le graphe est-il orienté ?
    sortie : aucune, mais création d'un fichier nom.png
    """
    # 1 creation de l'objet 
    if orient :
        g = Digraph(format='png')   # graphe avec des arcs
    else :
        g = Graph(format='png')   # graph avec des arêtes
    for cle in gr.keys(): # sommets
        g.node(cle,cle)
    for cle,valeur in gr.items(): # arêtes
        for s in valeur:
            if orient or cle<=s : # astuce pour éviter les doublons dans les arêtes 
                g.edge(cle,s)
    g.render(nom)

## II - Le réseau social

- A est ami avec B, C et D
- B est ami avec A et D
- C est ami avec A, E et D
- D est ami avec tous les autres abonnés
- E est ami avec C, D et F
- F est ami avec E et D

### a) par listes d'adjacence :

In [2]:
# implémentation par liste d'adjacence
reseau1 = {'A':['B','C','D'],
           'B':['A','D'],
           'C':['A','E','D'],
           'D':['A','B','C','E','F'],
           'E':['C','D','F'],
           'F':['E','D'] }

In [18]:
affiche_adj(reseau1,"cas1",False)

<img src="cas1.png">

In [5]:
# Quels sont les amis de E ?
print(reseau1['E'])

['C', 'D', 'F']


In [9]:
# De qui D est-il ami ? (ce code doit aussi fonctionner si les relations ne sont pas symétriques !)
liste=[]
for cle,valeur in reseau1.items():
    if 'D' in valeur:
        liste.append(cle)
print(liste)

['A', 'B', 'C', 'E', 'F']


### b) par matrice d'adjacence

In [11]:
sommets=['A','B','C','D','E','F']
adjacence= [[0,1,1,1,0,0],
            [1,0,0,1,0,0],
            [1,0,0,1,1,0],
            [1,1,1,0,1,1],
            [0,0,1,1,0,1],
            [0,0,0,1,1,0]]
reseau2={"sommets":sommets,"adjacence":adjacence}

In [21]:
affiche_mat(reseau2,"cas2bis",False)

<img src="cas2bis.png">

In [26]:
# Quels sont les amis de E ?
liste=[]
for i in range(len(reseau2["sommets"])):
    if reseau2["adjacence"][4][i] == 1: # ligne 4 (E) colonne i
            liste.append(reseau2["sommets"][i])
print(liste)

['C', 'D', 'F']


In [27]:
# réciproquement de qui E est-il ami (on suppose que les relations ne sont pas forcément symétriques)
liste=[]
for i in range(len(reseau2["sommets"])):
    if reseau2["adjacence"][i][4] == 1:  # ligne i colonne 4 (E)
            liste.append(reseau2["sommets"][i])
print(liste)

['C', 'D', 'F']


### c : Passage d'une représentation à l'autre (difficile)

In [22]:
def delisteamatrice(g):
    """ entree : (dict) graphe définit par listes d'adjacences
        sortie : (dict) un graphe défini par matrice
                 ce graphe est un dictionnaire contenant 2 clés
                 "sommets" a pour valeur la liste des sommets
                 "adjacence" est la matrice d'adjacence
      """
    s=list(g.keys()) #liste des sommets
    n=len(s)  # ordre du graphe
    a=[[0]*n for i in range(n)]  #tableau a deux dimensions nxn
    for l in range(n): # s[l] sommet de depart
        for c in range(n): #s[c] sommet d'arrivee
            if s[c] in g[s[l]]:  # s[c] est-il dans les successeurs de s[l]
                a[l][c]=1
    return {"sommets":s,"adjacence":a}

In [24]:
g={'A':['B','C'],'B':['C'],'C':['A']}

In [25]:
delisteamatrice(g)

{'sommets': ['A', 'B', 'C'], 'adjacence': [[0, 1, 1], [0, 0, 1], [1, 0, 0]]}

In [38]:
def dematricealiste(g):
    """ entree : g (dict) un graphe défini par matrice
                 ce graphe est un dictionnaire contenant 2 clés
                 "sommets" a pour valeur la liste des sommets
                 "adjacence" est la matrice d'adjacence
    sortie : (dict) graphe définit par listes d'adjacences"""
    mondict={} # initialisation
    n = len(g["sommets"]) # ordre du graphe
    for l in range(n): # on parcourt les lignes
        successeurs=[] # initialisation de la liste des successeurs
        for c in range(n): # on parcourt les colonnes
            if g["adjacence"][c][l]==1:
                successeurs.append(g["sommets"][c]) # ajout d'un successeur
            mondict[g["sommets"][l]]=successeurs
    return mondict

In [39]:
g={'sommets': ['A', 'B', 'C'], 'adjacence': [[0, 1, 1], [0, 0, 1], [1, 0, 0]]}

In [40]:
dematricealiste(g)

{'A': ['C'], 'B': ['A'], 'C': ['A', 'B']}